In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nt = 50000
w = np.zeros(nt)
bu = np.zeros(nt)
bs = np.zeros(nt)
dt = 0.1
Nu = 0.012      # Holton pp 58
Ns = 0.0012

w[0] = 0.1
bu[0] = 0
bs[0] = 1


# leap frog
for t in range(1):
    if bs[t] > bu[t]:
        w[t+1] = w[t] + bs[t]*dt
    else:
        w[t+1] = w[t] + bu[t]*dt
    
    bu[t+1] = bu[t] - (Nu**2)*w[t]*dt
    bs[t+1] = bs[t] - (Ns**2)*w[t]*dt
    


for t in range(1,nt-1):
    if bs[t] > bu[t]:
        w[t+1] = w[t-1] + bs[t]*2.*dt
    else:
        w[t+1] = w[t-1] + bu[t]*2.*dt
    
    bu[t+1] = bu[t-1] - (Nu**2)*w[t]*2.*dt
    bs[t+1] = bs[t-1] - (Ns**2)*w[t]*2.*dt
    
M = bu[0]/Nu**2 - bs[0]/Ns**2

In [ ]:
plt.plot(w)

In [ ]:
plt.plot(bu, label='bu')
plt.plot(bs, label='bs')

### TLM

In [ ]:
# basic state?
w = np.zeros(nt)
bu = np.zeros(nt)
bs = np.zeros(nt)

# p for prime, perturbation
wp = np.zeros(nt)
bup = np.zeros(nt)
bsp = np.zeros(nt)

# initial condition for basic state
#w[0] = 0.1
#bu[0] = 0
#bs[0] = 0

# initial condition for perturbaitons are all zeros, 
#wp[0] = 1e-4
#bup[0] = 1e-5
#bsp[0] = 1e-6

# initial condition for basic state
w[0] = 0.1
bu[0] = 0
bs[0] = 0

# initial condition for perturbaitons are all zeros, 
wp[0] = 0.1
bup[0] = 0
bsp[0] = 0.1

# leap frog
for t in range(1):
    if bs[t] > bu[t]:
        wp[t+1] = wp[t] + dt*bup[t]
        w[t+1] = w[t] + bs[t]*dt
    else:
        wp[t+1] = wp[t] + dt*bsp[t]
        w[t+1] = w[t] + bu[t]*dt
        
    bup[t+1] = bup[t] - (Nu**2)*wp[t]*dt
    bu[t+1] = bu[t] - (Nu**2)*w[t]*dt
    bsp[t+1] = bsp[t] - (Ns**2)*wp[t]*dt
    bs[t+1] = bs[t] - (Ns**2)*w[t]*dt
    


for t in range(1,nt-1):
    if bs[t] > bu[t]:
        w[t+1] = w[t-1] + bs[t]*2.*dt
        wp[t+1] = wp[t-1] + bsp[t]*2.*dt
    else:
        w[t+1] = w[t-1] + bu[t]*2.*dt
        wp[t+1] = wp[t-1] + bup[t]*2.*dt
    
    bu[t+1] = bu[t-1] - (Nu**2)*w[t]*2.*dt
    bup[t+1] = bup[t-1] - (Nu**2)*wp[t]*2.*dt
    bs[t+1] = bs[t-1] - (Ns**2)*w[t]*2.*dt
    bsp[t+1] = bsp[t-1] - (Ns**2)*wp[t]*2.*dt
tlm_pert = [wp, bup, bsp]


In [ ]:
def sens_subplots(var, ax, title):
    cs = ax.plot(var)
    ax.set_title(title)

In [ ]:
nrows = 1
ncols = 3
fig,ax = plt.subplots(nrows=nrows,
                      ncols=ncols, figsize=(16,5))
sens_subplots(wp, ax[0], "leap frog TLM w'")
sens_subplots(bup, ax[1], "bu'")
sens_subplots(bsp, ax[2], "bs'")

In [ ]:
wp[-1], bup[-1], bsp[-1]

In [ ]:
a = pd.read_csv('/home1/class/fall17/nchen67/moistPV/fortran_moist_leapfrog/fort.16', delim_whitespace=True, 
                header=None, names=['a_w', 'a_bu', 'a_bs'])
a_w = a.a_w.values[::-1]
a_bu = a.a_bu.values[::-1]
a_bs = a.a_bs.values[::-1]

In [ ]:
nrows = 1
ncols = 3
fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12,5))
sk = 10
sens_subplots(a_w[::sk], ax[0], 'sensitivity to w')
sens_subplots(a_bu[::sk], ax[1], 'sensitivity to bu')
sens_subplots(a_bs[::sk], ax[2], 'sensitivity to bs')

In [ ]:
plt.figure(figsize=(18,4))
sk=1
plt.plot(a_w[0:100:sk])

In [ ]:
a = pd.read_csv('/home1/class/fall17/nchen67/moistPV/fortran_moist_leapfrog/fort.15', delim_whitespace=True, 
                header=None, names=['w_prime', 'bu_prime', 'bs_prime'])
w_prime = a.w_prime
bu_prime = a.bu_prime
bs_prime = a.bs_prime

In [ ]:
nrows = 1
ncols = 3
fig,ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(12,5))
sens_subplots(w_prime, ax[0], 'w')
sens_subplots(bu_prime, ax[1], 'bu')
sens_subplots(bu_prime, ax[2], 'bs')

In [ ]:
w_prime.size, a_w.size

In [ ]:
dR = w_prime*a_w[1:] + bu_prime*a_bu[1:] + bs_prime*a_bs[1:]
plt.plot(dR)
